In [ ]:
class WorstOf():
    def __init__(self, type, spots, vols, corr_matrices, r, T, strike, num_of_simulations = 100000):
        self.type = type
        self.spots = spots
        self.vols = vols
        self.corr_matrices = corr_matrices
        self.r = r
        self.T = T
        self.strike = strike
        self.num_of_simulations = num_of_simulations



    def simulateGeoPaths(self):
        """
        Simulation of multiple underlying assets under the 
        risk-neutral measure using a single-step Monte Carlo.
        """

        n_assets = len(self.spots)
        L = np.linalg.cholesky(self.corr_matrices)        # Cholesky decomposition of the correlation matrix
        Z = np.random.normal(0.0, 1.0, (self.num_of_simulations, n_assets))
        correlated_Z = np.dot(Z, L.T)
        #correlated_Z = Z @ L
        single_step_terminal_price = np.zeros_like(correlated_Z)

        for i in range(n_assets):
            drift = (self.r - 0.5 * self.vols[i]**2) * self.T
            diffusion = self.vols[i] * np.sqrt(self.T) * correlated_Z[:, i]
            single_step_terminal_price[:, i] = self.spots[i] * np.exp(drift + diffusion)
        return single_step_terminal_price
    
    def price(self):
        """
        Price the worst of option using the simulated price paths.
        """
        
        price_Paths = self.simulateGeoPaths()
        percentage_performance = price_Paths / self.spots
        worst_performer_index = np.argmin(percentage_performance, axis=1)
        worst_performer_dollar = np.array([percentage_performance[i, idx] * self.spots[idx] for i, idx in enumerate(worst_performer_index)])
        if self.type == 'Call':
            payoff = np.maximum(0.0, worst_performer_dollar - self.strike*self.spots[worst_performer_index])
        elif self.type == 'Put':
            payoff = np.maximum(0.0, self.strike*self.spots[worst_performer_index] - worst_performer_dollar)
        
        discount_payoff = payoff * np.exp(-self.r * self.T)
        return np.mean(discount_payoff)
